YADDUM test

In [1]:
import YADDUM as YADDUM
import numpy as np  
import matplotlib.pyplot as plt
import xarray as xr

In [2]:
lidar_uncertainty = YADDUM.Uncertainty()

In [3]:
lidar_uncertainty.add_measurements('mesh', 'horizontal_mesh', 
                                   resolution = 10, 
                                   mesh_center = np.array([0,0,100]), 
                                   extent = 1000)

lidar_uncertainty.add_measurements('pts', 'points', 
                                  positions = np.array([[500,-500,100], [1000,2,300]]))

Measurements 'mesh' added to the measurement dictionary, which now contains 1 measurement instance(s).
Measurements 'pts' added to the measurement dictionary, which now contains 2 measurement instance(s).


In [4]:
model_pars={'wind_speed':10,
            'upward_air_velocity':0,
            'wind_from_direction':0,
            'reference_height':100,
            'shear_exponent':0.2}
lidar_uncertainty.add_atmosphere('pl_1', 'power_law', model_pars)

Atmosphere 'pl_1' added to the atmosphere dictionary, which now contains 1 atmosphere instance(s).


In [5]:
uncertainty_pars = {'U_los':0.1,
                    'U_azimuth':0.1,
                    'U_elevation':0.1, 
                    'U_range':1}

lidar_pos_1 = np.array([0,0,0])
lidar_pos_2 = np.array([1000,1000,0])

lidar_uncertainty.add_instrument('koshava', lidar_pos_1, **uncertainty_pars)
lidar_uncertainty.add_instrument('whittle', lidar_pos_2, **uncertainty_pars)

Instrument 'koshava' of category 'lidar' added to the instrument dictionary, which now contains 1 instrument(s).
Instrument 'whittle' of category 'lidar' added to the instrument dictionary, which now contains 2 instrument(s).


In [6]:
lidar_uncertainty.calculate_uncertainty(['koshava', 'whittle'],'mesh','pl_1', 'dual-Doppler')

/Volumes/Secondary_Drive/work/projects/dual-Doppler-uncertainty-model/YADDUM/_YADDUM.py:895: RuntimeWarning: divide by zero encountered in reciprocal
  uncertainty =((wind_speed * (np.sin(angle_dif))**2)**-1 *
/Volumes/Secondary_Drive/work/projects/dual-Doppler-uncertainty-model/YADDUM/_YADDUM.py:931: RuntimeWarning: divide by zero encountered in reciprocal
  (wind_speed**4 * np.sin(angle_dif)**2)**-1


In [7]:
lidar_uncertainty.uncertainty

<xarray.Dataset>
Dimensions:                          (Easting: 201, Id: 2, Northing: 201)
Coordinates:
  * Id                               (Id) object 'koshava' 'whittle'
  * Easting                          (Easting) int64 -1000 -990 ... 990 1000
  * Northing                         (Northing) int64 -1000 -990 ... 990 1000
    Height                           int64 100
Data variables:
    azimuth                          (Id, Northing, Easting) float64 225.0 ... 0.0
    elevation                        (Id, Northing, Easting) float64 4.045 ... 90.0
    range                            (Id, Northing, Easting) float64 1.418e+03 ... 100.0
    radial_speed                     (Id, Northing, Easting) float64 -7.053 ... 6.123e-16
    azimuth_gain                     (Id, Northing, Easting) float64 -0.7053 ... 0.0
    elevation_gain                   (Id, Northing, Easting) float64 -1.945 ... -1.0
    range_gain                       (Id, Northing, Easting) float64 -9.95e-05 ... 1.225e-19


In [ ]:
azimuth_1 = self.uncertainty.azimuth.sel(Id = 'koshava').values
azimuth_2 = self.uncertainty.azimuth.sel(Id = 'whittle').values

In [8]:
lidar_uncertainty.wind_field.wind_speed.values

array([[10., 10., 10., ..., 10., 10., 10.],
       [10., 10., 10., ..., 10., 10., 10.],
       [10., 10., 10., ..., 10., 10., 10.],
       ...,
       [10., 10., 10., ..., 10., 10., 10.],
       [10., 10., 10., ..., 10., 10., 10.],
       [10., 10., 10., ..., 10., 10., 10.]])

In [ ]:
nrows = lidar_uncertainty.measurements['mesh']['nrows']
ncols = lidar_uncertainty.measurements['mesh']['ncols']
prob_cord = lidar_uncertainty.probing_dict['koshava'].reshape(nrows, ncols,3)
rad_speed = lidar_uncertainty.radial_vel_dict['koshava'].reshape(nrows, ncols)
coords = lidar_uncertainty.measurements['mesh']['positions']

test = xr.Dataset({'temp':(['Northing', 'Easting'], lidar_uncertainty.wind_direction_uncertainty)},
            
                             coords={'Easting': np.unique(coords[:,0]), 'Northing': np.unique(coords[:,1])}
                            )

In [ ]:
test

In [ ]:
lidar_uncertainty.wind_field.wind_from_direction.values.reshape(-1)

In [ ]:
lidar_uncertainty.wind_field.attrs['model_parameters']['shear_exponent']

In [ ]:
MY_DPI = 96
FONT_SIZE = 16
STEP = 1
MAP_EXTENT = 500


lidar_uncertainty = YADDUM.LidarUncertainty()


model_pars={'horizontal_speed':10,
                  'vertical_speed':0,
                  'wind_direction':0,
                  'reference_height':100,
                  'shear_exponent':0.2}

uncertainty_pars = {'U_los':0.1,
                    'U_azimuth':0.1,
                    'U_elevation':0.1, 
                    'U_range':1}      

# lidar_pos_1 = np.array([128.5, -1924,0]) # absolute position {446080, 6259660} 
# lidar_pos_2 = np.array([-128.5, 1924, 0]) # absolute position {445823, 6263508}
lidar_pos_1 = np.array([0,0,0])


_,_,meas_pts = YADDUM.generate_mesh(lidar_pos_1 + np.array([0, 0,100]), MAP_EXTENT, STEP)

meas_pts = np.array([
    [0,1000, 100],
    [1000,1000, 100],
    [1000,0, 100],
    [1000,-1000, 100],    
    [0,-1000,100],
    [-1000,-1000,100],
    [-1000,0,100],
    [-1000,1000,100]
])


lidar_uncertainty.add_atmosphere('pl_1', 'power_law', model_pars)
lidar_uncertainty.add_measurements('grid', meas_pts, atmosphere_id = 'pl_1')


lidar_uncertainty.add_lidar('koshava', lidar_pos_1, **uncertainty_pars)
# lidar_uncertainty.add_lidar('whittle', lidar_pos_2, **uncertainty_pars)

lidar_uncertainty.calculate_uncertainty(['koshava'], 'grid', 'pl_1')



In [ ]:
coords = lidar_uncertainty.lidars['koshava']['probing']['coordinates']
uncert = lidar_uncertainty.lidars['koshava']['calculated_uncertainty']['elevation_gain']

In [ ]:
da = xr.DataArray(data=uncert.reshape(1,len(uncert)),
                  coords={"azimuth": (["x","y"], coords[:,0].reshape(1,len(coords[:,0]))),
                          "elevation": (["x","y"], coords[:,1].reshape(1,len(coords[:,1]))),
                          "range": (["x","y"], coords[:,2].reshape(1,len(coords[:,2])))},
                  dims=["x","y"])

In [ ]:
da = xr.DataArray(data=uncert,
                  coords={"azimuth": (["point"], coords[:,0]),
                          "elevation": (["point"], coords[:,1]),
                         "range": (["point"], coords[:,2])},
                  dims=["point"])

In [ ]:

ax = plt.subplot()
da.plot(marker='o')
ax.scatter(4, 0)

In [ ]:
# some example from WEB

np.random.seed(123)
shape = (1,4)    # needs to be 2-dimensional, could be (2,2), (4,1)

# all three arrays have the same shape
temperature = 15 + 10 * np.random.randn(*shape)
lat = np.random.uniform(low=-90, high=90, size=shape)
lon = np.random.uniform(low=-180, high=180, size=shape)

# round to two digits after decimal point
temperature, lat , lon = np.around([temperature, lat, lon], decimals=2)

da = xr.DataArray(data=temperature,
                  coords={"lat": (["x","y"], lat),
                          "lon": (["x","y"], lon)},
                  dims=["x","y"])

In [ ]:
tmp = {'mesh_center', 'extent', 'resolution'}

In [ ]:
tmp2 = {'resolution','mesh_center', 'extent'}

In [ ]:
tmp2 == tmp